In [36]:
#讀取資料
train_df=pd.read_csv("C:/Users/user/Desktop/train.csv")
test_df=pd.read_csv("C:/Users/user/Desktop/test.csv")

In [ ]:
#資料前處理
#categorical features
categorical = train_df.select_dtypes(include =[np.object])
print("Categorical Features in DataSet:",categorical.shape[1])
print(categorical.columns)
#numerical features
numerical= train_df.select_dtypes(include =[np.float64,np.int64])
print("Numerical Features in DataSet:",numerical.shape[1])
print(numerical.columns)



# let's check for missing values
train_df.isnull().sum()
test_df.isnull().sum()


In [39]:
def preprocessing(train_df):
    label = train_df["is_claim"]
    train_df = train_df.drop(["policy_id", "is_claim"], axis=1)
    # numerical feature -> categorical feature (discreti)


    # train.csv -> train, validation, test
    # split test using index suffle or index sample
    # suffle first
    # index = list(range(len(train_df)))
    index = np.arange(len(train_df))
    random.seed(1)
    random.shuffle(index) # inplace
    # split by proportion 8:2
    split = np.ceil(len(train_df)*0.8).astype(int)
    train_x = train_df.loc[index[:split],]
    test_x = train_df.loc[index[split:]]
    train_y = label.loc[index[:split],]
    test_y = label.loc[index[split:]]
    # split validation

    return train_x, test_x, train_y, test_y
train_df = pd.read_csv("C:/Users/user/Desktop/train.csv")
train_x, test_x, train_y, test_y = preprocessing(train_df)

In [42]:
#Classification Task
#i. Naïve Bayer classifier
import numpy as np
import pandas as pd
import math


def norm(val,mean,std):
    pdf = 1/math.sqrt(2*math.pi*std) * math.exp(math.pow(val-mean,2)/(2*std*std))
    return pdf

def Naive_bayes(train:pd.DataFrame):
    label = train.columns.values[-1]
    feature = train.columns.values[:-1]
    parameters = {}
    label_value = train[label].unique()
    parameters[label] = {}
    for val in label_value:
        D_C = len(train[train[label] == val])
        D = len(train)
        N = len(label_value)
        parameters[label][val] = (D_C + 1)/N+D

    for fea in feature:
        if fea not in parameters.keys():
            parameters[fea] = {}
        # print("int" in str(train[fea].dtype))
        if ("object" in str(train[fea].dtype)) or ("int" in str(train[fea].dtype)):
            feature_value = train[fea].unique()
            N_i = len(feature_value)
            for feature_val in feature_value:
                parameters[fea][feature_val] = {}
                for label_val in label_value:
                    # print(fea,feature_val,label_val)
                    D_ci = len(train.loc[(train[label] == label_val) & (train[fea] == feature_val)])
                    D_c = len(train[train[label] == label_val][fea])
                    # print((D_ci+1)/(N_i+D_c))
                    parameters[fea][feature_val][label_val] = (D_ci+1)/(N_i+D_c)
            #         break
            #     break
            # break
        else:
            for label_val in label_value:
                parameters[fea][label_val] = {}
                mean_f = train[train[label] == label_val][fea].mean()
                std_f = train[train[label] == label_val][fea].std()
                parameters[fea][label_val]["mean"] = mean_f
                parameters[fea][label_val]["std"] = std_f

    return parameters

def predic(test:pd.DataFrame,label,parameters:{}):
    predic_label = []
    feature = test.columns.values
    # print(feature)
    label_n = label.columns.values
    label_values = label[str(label_n[0])].unique()
    max_p = -999

    res = ""
    p = 0

    for value in test.index:
        for label_val in label_values:
            p = parameters[str(label_n[0])][label_val]
            for fea in feature:

                if ("object" in str(test[fea].dtype)) or ("int" in str(test[fea].dtype)):
                    p *= parameters[fea][test.loc[value,fea]][label_val]
                else:
                    p *= norm(test.loc[value,fea],mean = parameters[fea][label_val]["mean"],std = parameters[fea][label_val]["std"])

            if p > max_p:
                res = label_val
                max_p = p
            predic_label.append(res)

    return  predic_label


def train_x_train_y(data,wanna_test = False,test_num = 0.25):

    data1 = data.iloc[np.random.permutation(len(data))]
    if wanna_test :

        data2 = data1[0:round((1-test_num)*len(data))]
        train_y = data2.iloc[:,-1]
        train_x = data2.drop(str(train_y.name), axis = 1)
        data3 = data1[round((1-test_num)*len(data)):]
        test_y = data3.iloc[:,-1]
        test_x = data3.drop(str(test_y.name), axis = 1)

        return train_x,train_y,test_x,test_y
    else:
        train_y = data1.iloc[:, -1]
        train_x = data1.drop(str(train_y.name), axis = 1)
        return train_x, train_y

def confusion_matrix(result,test_y):
    TP,TN,FT,FN = 0,0,0,0
    print(result[1])
    test_y = np.asarray(test_y)
    print(test_y[1])
    for i in range(len(result)):
        if result[i] > 0 :
            if result[i] == test_y[i]:
                TP += 1
            else:
                TN += 1
        else:
            if result[i] == test_y[i]:
                FT += 1
            else:
                FN += 1
    accuracy = (TP + FT) / len(result)
    cm = np.array([[TP,FN],[TN,FT]])
    print("confusion_matrix:")
    print(cm)
    print("accuracy:",accuracy)

# reference https://blog.csdn.net/CarryLvan/article/details/109236906

if __name__ == '__main__':
    data3 = pd.read_csv("train.csv")
    data3 = data3.drop(columns="policy_id")
    train_x, train_y, test_x, test_y = train_x_train_y(data3, wanna_test=True)
    train_x_y = pd.concat([train_x, train_y], axis=1)
    model_naive = Naive_bayes(train_x_y)
    # print(model_naive["population_density"][8794][0])
    # print(type(test_y))
    # print(test_x)
    result = predic(pd.DataFrame(test_x),pd.DataFrame(test_y),model_naive)
    print(result)
    print(train_y)
    confusion_matrix(result,train_y)

In [48]:
#ii. Random Forest Classifier
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import operator
import pickle
import random


def transfrom_to_train(data: pd.DataFrame):
    data_ = data.to_numpy().tolist()
    attributeList = data.columns.values.tolist()

    return data_, attributeList
class decision_tree(object):


    def GetClassInfo(self,Dataset):
        ClassInfo = {}
        for item in Dataset:
            # check答案项目
            if item[-1] not in ClassInfo.keys():
                ClassInfo[item[-1]] = 1
            else:
                ClassInfo[item[-1]] += 1
            classInfo = dict(sorted(ClassInfo.items(), key=operator.itemgetter(1), reverse=True))
        return classInfo

    def maxclass(self,classinfo):
        max = list(classinfo.keys())[0]
        return max

    def compute_entropy(self,Dataset):
        classinfo  = self.GetClassInfo(Dataset)
        Ent = 0
        amount = 0
        p = []

        for _,value in classinfo.items():
            p.append(value)
            amount += value
        for p_k in p :
            Ent -= p_k/amount*np.log(p_k/amount)

        return Ent

    def getgainNPartition(self,Dataset,featureindex,featurelist):
        gain = self.compute_entropy(Dataset)
        lenth_dataset = len(Dataset)

        NPartition = {}
        for dataitem in Dataset:

            if dataitem[featureindex] not in NPartition.keys():
                NPartition[dataitem[featureindex]] = [dataitem]
            else:
                NPartition[dataitem[featureindex]].append(dataitem)
        # if featurelist[featureindex] == "make":
        #     print(NPartition)
        # print(NPartition)
        lenth = []
        Ent = []
        amount = 0

        for _,subdataset in NPartition.items():
            Ent.append(self.compute_entropy(subdataset))
            lenth.append(len(subdataset))
            amount += len(subdataset)
        for i in range(len(Ent)):
            gain -= lenth[i]/lenth_dataset * Ent[i]

        return gain,NPartition

    def CreateDecisionTree(self,Dataset,featurelist):
        lenth_dataset = len(Dataset)
        classinfo = self.GetClassInfo(Dataset)
        Tree = {}
        #给定的属性集为空 ---- 不能划分
        if len(featurelist) == 0:
            # print(self.maxclass(classinfo))
            return self.maxclass(classinfo)

        #给定的数据集所有label都相同 ---- 无需划分
        for key,value in classinfo.items():
            if value == lenth_dataset:
                return key
                break
        #样本在属性集上取值都相等 ---- 无法划分
        temp = Dataset[0][:-1]
        sment = 0
        for dataitem in Dataset[:-1]:
            if dataitem == temp:
                sment += 1
        if sment == lenth_dataset:

            return self.maxclass(classinfo)
        # 选择最佳划分属性
        bestIdex = 0
        bestGain = 0
        bestNPartition = {}

        for indedx in range(len(featurelist)):
            gain,NPartition = self.getgainNPartition(Dataset,indedx,featurelist)
            if gain > bestGain:
                bestIdex = indedx
                bestGain = gain
                bestNPartition = NPartition

        attr_Name = featurelist[bestIdex]
        # print(type(featurelist))
        del featurelist[bestIdex]

        for _,vallist in bestNPartition.items():
            for i in range(len(vallist)):
                temp = vallist[i][:bestIdex]
                temp.extend(vallist[i][bestIdex+1:])
                vallist[i] = temp
        # # 为了方便后面建子树，将此时的attr对应的那列去除
        # 根据属性的值，建立分叉节点
        # if bestNPartition.values() == []:
        #     break

        Tree[attr_Name] = {}
        if not bestNPartition:
            # print(bestNPartition.values())
            return self.maxclass(classinfo)
        # print(featurelist)
        for keyAttrVal, valDataset in bestNPartition.items():

            # 因为python对iterable list对象的传参是按地址传参，会改变attributeList的值
            # 所以在传attributeList参数的时候，创建一个副本，就相当于按值传递了
            subLabels = featurelist[:]

            # valDataset是已去除attr的data，attributeList是已去除attr的attributeList
            Tree[attr_Name][keyAttrVal] = self.CreateDecisionTree(valDataset, subLabels)
        return Tree

    def vote(self,predic_label):
        vote_dict = {}
        for result in predic_label:

            if result not in vote_dict.keys():
                vote_dict[result] = 1
            else:
                vote_dict[result] += 1

            sort_vote_disc = sorted(vote_dict.items(), key=operator.itemgetter(1), reverse=True)
            return sort_vote_disc[0][0]



    def Predict(self,DataSet, testArrtList, decisionTree):
        predicted_label = []

        for dataItem in DataSet:
            cur_decisionTree = decisionTree
            # 如果root就是叶子结点leaf
            if type(cur_decisionTree) == set:  # 例如：{'N'}
                node = list(cur_decisionTree)
            else:
                node = list(cur_decisionTree.keys())[0]
                # 只要temp处在attributeList，说明当前处在树枝结点(非叶子)上, 否则处在叶子结点
                while node in testArrtList:

                    try:

                        cur_index = testArrtList.index(node)  # 0 2
                        cur_element = dataItem[cur_index]  # 0 0
                        cur_decisionTree = cur_decisionTree[node][cur_element]  # {'student': {0: 'N', 1: 'Y'}} N
                        # print(cur_decisionTree)
                    except:

                        root = list(cur_decisionTree[node].keys())
                        # print(root)
                        cur_element = self.vote(root)
                        cur_decisionTree = cur_decisionTree[node][cur_element]


                    if type(cur_decisionTree) == dict:
                        node = list(cur_decisionTree.keys())[0]  # student
                    else:
                        node = cur_decisionTree


            # print(node)
            predicted_label.append(node)
        return predicted_label





class random_forest():

    def bagging(self,dataset,sample_num,dataset_size,typ = "None"):

        subdata_list = []
        if typ == "None":
            for datalist in range(sample_num):
                sub = random.sample(dataset,dataset_size)
                print(len(sub[0]),len(sub))
                # print(sub[0][5])
                subdata_list.append(sub)
            return subdata_list
        else :
            if typ == "time_series":
                pass

    def vote(self,predic_label):
        vote_dict = {}
        for result in predic_label:
            if result not in vote_dict.keys():
                vote_dict[result] = 1
            else:
                vote_dict[result] += 1
        sort_vote_disc = sorted(vote_dict.items(), key=operator.itemgetter(1), reverse=True)

        return sort_vote_disc[0][0]

    def fit(self,dataset,featurelist,num = 100,dataset_size = 500,typ = "None"):
        self.model_list = []
        # tree = decision_tree()
        bagging_dataset = self.bagging(dataset,num,dataset_size,typ)
        for data in bagging_dataset:
            tree = decision_tree()
            model = tree.CreateDecisionTree(data,featurelist)
            self.model_list.append(model)


    def predic(self,test_data):
        predic_label = []
        predic_result =[]
        predic_dict = []
        test_,attrib = transfrom_to_train(test_data)
        # print(attrib)
        for model in self.model_list:
            # print(model.items())

            label = decision_tree().Predict(test_,attrib,model)
            # print(label)
            predic_label.append(label)
        # print(predic_label)
        # print(len(predic_label[0]),len(predic_label))
        for j in range(len(predic_label[0])):
            for i in range(len(predic_label)):
                # print(i,j)
                predic_dict.append(predic_label[i][j])
            predic_result.append(self.vote(predic_dict))

        return predic_result


def train_x_train_y(data,wanna_test = False,test_num = 0.25):

    data1 = data.iloc[np.random.permutation(len(data))]
    if wanna_test :

        data2 = data1[0:round((1-test_num)*len(data))]
        train_y = data2.iloc[:,-1]
        train_x = data2.drop(str(train_y.name), axis = 1)
        data3 = data1[round((1-test_num)*len(data)):]
        test_y = data3.iloc[:,-1]
        test_x = data3.drop(str(test_y.name), axis = 1)

        return train_x,train_y,test_x,test_y
    else:
        train_y = data1.iloc[:, -1]
        train_x = data1.drop(str(train_y.name), axis = 1)
        return train_x, train_y

def confusion_matrix(result,test_y):
    TP,TN,FT,FN = 0,0,0,0
    # print(result[1])
    test_y = np.asarray(test_y)
    # print(test_y[1])
    for i in range(len(result)):
        if result[i] > 0 :
            if result[i] == test_y[i] :
                TP += 1
            else:
                TN += 1
        else:
            if result[i] == test_y[i] :
                FT += 1
            else:
                FN += 1
    accuracy = (TP + FT) / len(result)
    cm = np.array([[TP,FN],[TN,FT]])
    print("confusion_matrix:")
    print(cm)
    print("accuracy:",accuracy)

#referrance https://www.bilibili.com/video/BV1MK4y1P7TB/

if __name__ == '__main__':
    data3 = pd.read_csv("train.csv")
    # print(data3.shape)
    # data3 = data3.drop(columns=["policy_id"])
    # data3 = data3.drop(columns=['policy_tenure'])
    train_x, train_y, test_x, test_y = train_x_train_y(data3, wanna_test=True)
    train_x_y = pd.concat([train_x, train_y], axis=1)
    train_,attributeList = transfrom_to_train(data=train_x_y)
    # print(attributeList)
    del attributeList[len(attributeList)-1]
    print(attributeList)
    randomforest = random_forest()
    model_forest = randomforest.fit(train_,attributeList,num=5,dataset_size = 10000)
    result = randomforest.predic(test_x)
    print(result)
    confusion_matrix(result,test_y)

In [ ]:
#iii. Random Forest Classifier(scikit-learning)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def train_x_train_y(data,wanna_test = False,test_num = 0.25):

    data1 = data.iloc[np.random.permutation(len(data))]
    if wanna_test :

        data2 = data1[0:round((1-test_num)*len(data))]
        train_y = data2.iloc[:,-1]
        train_x = data2.drop(str(train_y.name), axis = 1)
        data3 = data1[round((1-test_num)*len(data)):]
        test_y = data3.iloc[:,-1]
        test_x = data3.drop(str(test_y.name), axis = 1)

        return train_x,train_y,test_x,test_y
    else:
        train_y = data1.iloc[:, -1]
        train_x = data1.drop(str(train_y.name), axis = 1)
        return train_x, train_y

data = pd.read_csv("train.csv")
data = data.drop(columns="policy_id")
train_x,train_y,test_x,test_y  = train_x_train_y(data,wanna_test = True,test_num = 0.25)
train_x,train_y,test_x,test_y = train_x.to_numpy(),train_y.to_numpy(),test_x.to_numpy(),test_y.to_numpy()
# print(train_x.shape)
# train_x = train_x.reshape((41014,-1))
# print(train_x.shape)
# train_y = train_y.reshape((-1,1))
print(train_y.shape)
# train_y = np.reshape(arousal_lable,(624,1))
rfc = RandomForestClassifier(random_state=0,class_weight="balanced")
rfc = rfc.fit(train_x,train_y)
score_r = rfc.score(test_x,test_y)
pred = rfc.predict(test_x)
print("Random Forest:{}".format(score_r))

mat = confusion_matrix(test_y, pred)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False)
plt.xlabel('true label')
plt.ylabel('predicted label')
plt.show()


In [ ]:
#iv. XGBoost, Catboost, LightGBM (can use other public code sources).
#——————————————————— xgboost ———————————————————

import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
import pandas as pd
import matplotlib
from sklearn.metrics import mean_squared_error


data = pd.read_csv("train.csv")
data = data.drop(columns="policy_id")

y = data.iloc[:,-1]
X = data.drop(str(y.name), axis = 1)

x_train, x_predict, y_train, y_predict = train_test_split(X, y, test_size=0.10, random_state=100)
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=100)


dtrain = xgb.DMatrix(data=x_train,label=y_train,missing=-999.0)
dtest = xgb.DMatrix(data=x_test,label=y_test,missing=-999.0)

param = {'max_depth': 7, 'eta': 1, 'silent': 1, 'objective': 'reg:linear'}
param['nthread'] = 4
param['seed'] = 100
param['eval_metric'] = 'auc'


num_round = 10
evallist = [(dtest, 'eval'), (dtrain, 'train')]


bst_with_evallist_and_early_stopping_10 = xgb.train(param, dtrain, num_round*100, evallist,early_stopping_rounds=10)
dpredict = xgb.DMatrix(x_predict)
ypred_with_evallist_and_early_stopping_100 = bst_with_evallist_and_early_stopping_10.predict(dpredict,ntree_limit=bst_with_evallist_and_early_stopping_10.best_ntree_limit)
print(ypred_with_evallist_and_early_stopping_100)

print("RMSE of bst_with_evallist_and_early_stopping_100 ：", np.sqrt(mean_squared_error(y_true=y_predict,y_pred=ypred_with_evallist_and_early_stopping_100)))


# —————————————————— catboost ——————————————————————


from catboost import CatBoostClassifier,Pool

cat_features = [0, 1]  

x_train, x_predict, y_train, y_predict = train_test_split(X, y, test_size=0.10, random_state=100)
# print(x_train.shape)
# print(y_train.shape)
# 定義模型定義模型
train_data = Pool(x_train,y_train)
model = CatBoostClassifier(iterations=10, learning_rate=1, depth=2)

# 訓練
model.fit(train_data)

# 預測類別
preds_class = model.predict(x_predict)
print(preds_class)

# —————————————————— LightGBM ——————————————————————

import lightgbm as lgb


train_data = lgb.Dataset(data=x_train,label=y_train)
test_data = lgb.Dataset(data=x_test,label=y_test)

param = {'num_leaves':31, 'num_trees':100, 'objective':'regression'}
param['metric'] = 'rmse'

In [ ]:
# Cross-Validation
#i. Use k=3,5,10, and make some discussions of your observation.
import numpy as np
import math
import pandas as pd
from tqdm import tqdm
from Naivebeyes import Naive_bayes,train_x_train_y,confusion_matrix
from randomforest import random_forest
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier,Pool
import xgboost as xgb
import lightgbm as lgb


class K_fold_cross_validation(object):

    def __init__(self,k = 5,dataset = "None",model = "None"):
        """
        :type fit: class : fit
        :type predic: class : predic
        :type dataset: pd.Datafram
        """
        self.k = k
        self.data = dataset
        self.model = model


    def Average(self,lst):
        return sum(lst) / len(lst)

    def cut_data(self):
        df = self.data
        k = self.k
        df_num = len(df)
        df = df.iloc[np.random.permutation(len(df))]
        every_epoch_num = int((df_num / k))
        self.cut_reslut = []
        for index in (range(k)):#tqdm
            if index < k - 1:
                df_tem = df[every_epoch_num * index: every_epoch_num * (index + 1)]
            else:
                df_tem = df[every_epoch_num * index:]
            self.cut_reslut.append(df_tem)
        return self.cut_reslut
        # 原文链接：https: // blog.csdn.net / weixin_42599499 / article / details / 117809308
    def validation(self,need_label = False,is_xgb = False,is_lg = False,is_cat = False):
        self.cut_data()
        mean_accuracy, mean_precision, mean_recall, mean_F1 = [],[],[],[]
        for item in (range(self.k)):#tqdm
            datalist = self.cut_reslut
            vali_data = datalist[item]

            traindata = pd.DataFrame()
            for index in range(len(datalist)):
                if index == item:
                    continue
                else:
                    traindata = traindata.append(datalist[index])
            vali_y = vali_data.iloc[:, -1]
            vali_x = vali_data.drop(str(vali_y.name), axis=1)

            train_y = traindata.iloc[:, -1]
            train_x = traindata.drop(str(vali_y.name), axis=1)

            if is_xgb:
                param = {'max_depth': 7, 'eta': 1, 'silent': 1, 'objective': 'binary:logistic'}
                param['nthread'] = 4
                param['seed'] = 100
                param['eval_metric'] = 'auc'
                dtrain = self.model.DMatrix(data=train_x, label=train_y, missing=-999.0)
                dpredict = self.model.DMatrix(vali_x)
                # evallist = [(dtest, 'eval'), (dtrain, 'train')]
                self.model.train(param, dtrain, 1000,early_stopping_rounds=10)
                result = self.model.predict(dpredict)
            else:
                if is_lg:
                    train_data = self.model.Dataset(data=train_x, label=train_y)
                    test_data = self.model.Dataset(data=vali_x, label=vali_y)

                    param = {'num_leaves': 31, 'num_trees': 100, 'objective': 'binary'}
                    param['metric'] = 'rmse'

                    bst = self.model.train(param, train_data, 1000, valid_sets=[test_data])
                    result = bst.predict(test_data, num_iteration=bst.best_iteration)

                else:
                    if is_cat:
                        train_data = Pool(train_x, train_y)
                        self.model.fit(train_data)
                        result = self.model.predic(vali_x)
                    else:
                        self.model.fit(traindata)
                        if need_label:

                            result = self.model.predic(pd.DataFrame(vali_x), pd.DataFrame(vali_y))
                        else:
                            result = self.model.predic(vali_x)

            a1,a2,a3,a4 = confusion_matrix(result,vali_y,if_print=False)
            mean_accuracy.append(a1)
            mean_precision.append(a2)
            mean_recall.append(a3)
            mean_F1.append(a4)
        print('mean_accuracy: %s, mean_precision: %s, mean_recall: %s,mean_F1: %s' % (self.Average(mean_accuracy),self.Average(mean_precision),self.Average(mean_recall),self.Average(mean_F1)))





    def plot(self):
        pass


if __name__ == '__main__':
    data3 = pd.read_csv("train.csv")
    data3 = data3.drop(columns="policy_id")
    train_x, train_y, test_x, test_y = train_x_train_y(data3, wanna_test=True)
    train_x_y = pd.concat([train_x, train_y], axis=1)
    for k in [3,5,10]:
        # ------------- Naive bayes -------------
        # NB = Naive_bayes()
        # cv_N = K_fold_cross_validation(k=k, dataset = train_x_y,model=NB)
        # cv_N.validation(need_label=True)
        #------------- random forest -------------
        rfc = random_forest()
        cv_R = K_fold_cross_validation(k=k, dataset = train_x_y,model=rfc)
        cv_R.validation()
        #------------- sklearn random forest-------------
        srfc = RandomForestClassifier(random_state=0, class_weight="balanced")
        cv_SR = K_fold_cross_validation(k=k, dataset=train_x_y, model=srfc)
        cv_SR.validation()
        # ------------- XGboost -------------
        # model_XG =
        cv_XG = K_fold_cross_validation(k=k, dataset=train_x_y, model=xgb)
        cv_XG.validation(is_xgb=True)
        # ------------- catboost -------------
        model_cat = CatBoostClassifier(iterations=10, learning_rate=1, depth=2)
        cv_CAT = K_fold_cross_validation(k=k, dataset=train_x_y, model=model_cat)
        cv_CAT.validation(is_pool=True)
        #------------- LightGBM -------------
        # model_LG = lgb()
        cv_LG = K_fold_cross_validation(k=k, dataset=train_x_y, model=lgb)
        cv_XG.validation(is_lg=True)
        break




In [ ]:
#ii. Now you have a test dataset you have partitioned from train.csv. 
# Please design an algorithm that can merge/ aggregate the predicted resultsfrom k classifiers in k-fold cross-validation. Compare the performance and complexity of the cross-validation with Problem 1. 
import numpy as np
import math
import pandas as pd
from tqdm import tqdm
from Naivebeyes import Naive_bayes,train_x_train_y,confusion_matrix
from randomforest import random_forest
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier,Pool
import xgboost as xgb
import lightgbm as lgb


class K_fold_cross_validation(object):

    def __init__(self,k = 5,dataset = "None",model = "None"):
        """
        :type fit: class : fit
        :type predic: class : predic
        :type dataset: pd.Datafram
        """
        self.k = k
        self.data = dataset
        self.model = model


    def Average(self,lst):
        return sum(lst) / len(lst)

    def cut_data(self):
        df = self.data
        k = self.k
        df_num = len(df)
        df = df.iloc[np.random.permutation(len(df))]
        every_epoch_num = int((df_num / k))
        self.cut_reslut = []
        for index in (range(k)):#tqdm
            if index < k - 1:
                df_tem = df[every_epoch_num * index: every_epoch_num * (index + 1)]
            else:
                df_tem = df[every_epoch_num * index:]
            self.cut_reslut.append(df_tem)
        return self.cut_reslut
        # 原文链接：https: // blog.csdn.net / weixin_42599499 / article / details / 117809308
    def validation(self,test_x,test_y,need_label = False,is_xgb = False,is_lg = False,is_cat = False):
        self.cut_data()
        mean_accuracy, mean_precision, mean_recall, mean_F1 = [],[],[],[]
        for item in (range(self.k)):#tqdm
            datalist = self.cut_reslut
            vali_data = datalist[item]

            traindata = pd.DataFrame()
            for index in range(len(datalist)):
                if index == item:
                    continue
                else:
                    traindata = traindata.append(datalist[index])
            # vali_y = vali_data.iloc[:, -1]
            # vali_x = vali_data.drop(str(vali_y.name), axis=1)

            train_y = traindata.iloc[:, -1]
            train_x = traindata.drop(str(train_y.name), axis=1)

            if is_xgb:
                param = {'max_depth': 7, 'eta': 1, 'silent': 1, 'objective': 'binary:logistic'}
                param['nthread'] = 4
                param['seed'] = 100
                param['eval_metric'] = 'auc'
                dtrain = self.model.DMatrix(data=train_x, label=train_y, missing=-999.0)
                dpredict = self.model.DMatrix(test_x)
                # evallist = [(dtest, 'eval'), (dtrain, 'train')]
                xg = self.model.train(param, dtrain, 1000,early_stopping_rounds=10)
                result = xg.predict(dpredict)
            else:
                if is_lg:
                    train_data = self.model.Dataset(data=train_x, label=train_y)
                    test_data = self.model.Dataset(data=test_x, label=test_y)

                    param = {'num_leaves': 31, 'num_trees': 100, 'objective': 'binary'}
                    param['metric'] = 'rmse'

                    bst = self.model.train(param, train_data, 1000, valid_sets=[test_data])
                    result = bst.predict(test_data, num_iteration=bst.best_iteration)

                else:
                    if is_cat:
                        train_data = Pool(train_x, train_y)
                        self.model.fit(train_data)
                        result = self.model.predic(test_x)
                    else:
                        self.model.fit(traindata)
                        if need_label:

                            result = self.model.predic(pd.DataFrame(test_x), pd.DataFrame(test_y))
                        else:
                            result = self.model.predic(test_x)

            a1,a2,a3,a4 = confusion_matrix(result,test_y,if_print=False)
            mean_accuracy.append(a1)
            mean_precision.append(a2)
            mean_recall.append(a3)
            mean_F1.append(a4)
        print('mean_accuracy: %s, mean_precision: %s, mean_recall: %s,mean_F1: %s' % (self.Average(mean_accuracy),self.Average(mean_precision),self.Average(mean_recall),self.Average(mean_F1)))
if __name__ == '__main__':
    data3 = pd.read_csv("train.csv")
    data3 = data3.drop(columns="policy_id")
    train_x, train_y, test_x, test_y = train_x_train_y(data3, wanna_test=True)
    train_x_y = pd.concat([train_x, train_y], axis=1)
    for k in [5]:
        # ------------- Naive bayes -------------
        NB = Naive_bayes()
        cv_N = K_fold_cross_validation(k=k, dataset = train_x_y,model=NB)
        cv_N.validation(test_x,test_y,need_label=True)
        #------------- random forest -------------
        rfc = random_forest()
        cv_R = K_fold_cross_validation(k=k, dataset = train_x_y,model=rfc)
        cv_R.validation(test_x,test_y)
        #------------- sklearn random forest-------------
        srfc = RandomForestClassifier(random_state=0, class_weight="balanced")
        cv_SR = K_fold_cross_validation(k=k, dataset=train_x_y, model=srfc)
        cv_SR.validation(test_x,test_y)
        # ------------- XGboost -------------
        # model_XG =
        cv_XG = K_fold_cross_validation(k=k, dataset=train_x_y, model=xgb)
        cv_XG.validation(test_x,test_y,is_xgb=True)
        # ------------- catboost -------------
        model_cat = CatBoostClassifier(iterations=10, learning_rate=1, depth=2)
        cv_CAT = K_fold_cross_validation(k=k, dataset=train_x_y, model=model_cat)
        cv_CAT.validation(test_x,test_y,is_cat=True)
        #------------- LightGBM -------------
        # model_LG = lgb()
        cv_LG = K_fold_cross_validation(k=k, dataset=train_x_y, model=lgb)
        cv_XG.validation(test_x,test_y,is_lg=True)

In [ ]:
#iii. How do we know the performance of one model is really better than another one? Please compare the result in 5-fold cross-validation and the result of Problem 1 to justify which is “REALLY” better. Also show me why.